In [7]:
import cv2
import os

data_path = 'f:\\anaconda3\\lib\\site-packages\\cv2\\data'
data_path = r'D:\SoftInstall\Anacoda3\Lib\site-packages\cv2\data'

In [8]:
os.chdir(data_path)

In [9]:
ls *.xml

 驱动器 D 中的卷是 Data
 卷的序列号是 BC8A-BFB6

 D:\SoftInstall\Anacoda3\Lib\site-packages\cv2\data 的目录

2020/04/04  16:18           341,406 haarcascade_eye.xml
2020/04/04  16:18           601,661 haarcascade_eye_tree_eyeglasses.xml
2020/04/04  16:18           411,388 haarcascade_frontalcatface.xml
2020/04/04  16:18           382,918 haarcascade_frontalcatface_extended.xml
2020/04/04  16:18           676,709 haarcascade_frontalface_alt.xml
2020/04/04  16:18         2,689,040 haarcascade_frontalface_alt_tree.xml
2020/04/04  16:18           540,616 haarcascade_frontalface_alt2.xml
2020/04/04  16:18           930,127 haarcascade_frontalface_default.xml
2020/04/04  16:18           476,827 haarcascade_fullbody.xml
2020/04/04  16:18           195,369 haarcascade_lefteye_2splits.xml
2020/04/04  16:18            47,775 haarcascade_licence_plate_rus_16stages.xml
2020/04/04  16:18           395,322 haarcascade_lowerbody.xml
2020/04/04  16:18           828,514 haarcascade_profileface.xml
2020/04/04  16:18   

# 实时检测人脸

In [4]:
def crop_pic(img, x, y, w, h):
    cv2.imshow('face', img[x:x + w, y:y + h, :])

In [8]:
cap = cv2.VideoCapture(0) 
face_cascade = cv2.CascadeClassifier(os.path.join(data_path, 'haarcascade_frontalface_default.xml')) # 加载人脸特征库

In [9]:
while(True):
    ret, frame = cap.read() # 读取一帧的图像
    if not ret:
        print('failed to read camera data.')
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # 转灰

    faces = face_cascade.detectMultiScale(gray, scaleFactor = 3, minNeighbors = 5, minSize = (5, 5)) # 检测人脸
    for(x, y, w, h) in faces:
        crop_pic(frame, x, y, w, h)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2) # 用矩形圈出人脸
        
    cv2.imshow('Face Recognition', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release() # 释放摄像头
cv2.destroyAllWindows()

这里主要说下minNeighbors,minSize,maxSize.这三个参数.
通过这三个参数可以控制检测的精确度.
- minNeighbors 值越大,检测的准确度越高,不过耗时也越久.酌情调整.
- minSize 可以根据Screen 尺寸的一定比例来设置,别设置太小,不然会有一些错误干扰结果.
- maxSize 最大可检测尺寸,酌情调整.

# 动态区域检测

## 背景相减

### MOG2

In [29]:
import cv2

# 可选参数 比如 进行建模场景的时间长度 高斯混合成分的数量-阈值等
backSub = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
# backSub = cv2.createBackgroundSubtractorKNN()

# capture = cv.VideoCapture(cv.saqmples.findFileOrKeep(args.input))
capture = cv2.VideoCapture(0) 

while True:
    ret, frame = capture.read()
    if frame is None:
        break

    #update the background model
    fgMask = backSub.apply(frame)

    #get the frame number and write it on the current frame
    cv2.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv2.putText(frame, str(capture.get(cv2.CAP_PROP_POS_FRAMES)), (15, 15),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))

    #show the current frame and the fg masks
    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgMask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

capture.release() # 释放摄像头
cv2.destroyAllWindows()

## KNN

此算法结合了静态背景图像估计和每个像素的贝叶斯分割。
它使用前面很少的图像（默认为前 120 帧）进行背景建模。
使用了概率前景估计算法（使用贝叶斯估计鉴定前景）。
这是一种自适应的估计，新观察到的对象比旧的对象具有更高的权重，
从而对光照变化产生适应。一些形态学操作如开运算闭运算等被用来除去不需要的噪音。
在前几帧图像中你会得到一个黑色窗口。对结果进行形态学开运算对与去除噪声很有帮助。

```python
1.定义1个KNN背景分割器对象
2.定义视频对象
 
while True：
     
    3.一帧帧读取视频
    4.计算前景掩码
     
    5.二值化操作
    6.膨胀操作
 
    7.查找轮廓
    8.轮廓筛选
    9.画出轮廓（在原图像）
  
    10.显示图像帧，
```

### 简洁版

In [5]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
# 定义卷积核
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
fgbg = cv2.createBackgroundSubtractorKNN(dist2Threshold=True)

while (1):
    ret, frame = cap.read()
    if not ret:
        print('Input Error！输入错误！')
        break
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_ELLIPSE, kernel)

    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgmask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 动态区域检测1

In [2]:
import cv2

bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)

# 2.读取视频
camera = cv2.VideoCapture(0)

# 定义卷积核圆形,构建一个椭圆形的核
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

while True:
    ret, frame = camera.read()

    # 3. apply()函数计算了前景掩码
    fgmask = bs.apply(frame)

    # 4. 获得前景掩码（含有白色值以及阴影的灰色值），通过设定阈值将非白色（244~255）的所有像素都设为0，而不是1；
    th = cv2.threshold(fgmask.copy(), 244, 255, cv2.THRESH_BINARY)[1]  # 二值化操作
    # 5.膨胀操作
    dilated = cv2.dilate(th, kernel, iterations=2)  
    # 6. findContours函数参数说明: cv2.RETR_EXTERNAL只检测外轮廓，
    # cv2.CHAIN_APPROX_SIMPLE只存储水平，垂直，对角直线的起始点。
    contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 查找轮廓

    for c in contours:  # 从list列表取出每个轮廓
        if cv2.contourArea(c) < 400:  # 进行轮廓筛选 轮廓面积小于400 忽略
            continue

        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("mog", fgmask)
    cv2.imshow("thresh", th)
    cv2.imshow("detection", frame)

    if cv2.waitKey(100) & 0xff == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

### 动态区域检测2

In [19]:
import cv2


camera = cv2.VideoCapture(0)
history = 120  # 训练帧数

bs = cv2.createBackgroundSubtractorKNN(detectShadows=True)  # 背景减除器，设置阴影检测
bs.setHistory(history)

frames = 0

while True:
    res, frame = camera.read()
    if not res:
        print('Input Error！输入错误！')
        break

    # 获取 foreground mask
    fg_mask = bs.apply(frame)  

    if frames < history:
        frames += 1
        continue

    # 二值化
    th = cv2.threshold(fg_mask.copy(), 244, 255, cv2.THRESH_BINARY)[1]
    # 对原始帧进行膨胀去噪
    th = cv2.erode(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=2)
    dilated = cv2.dilate(th, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 3)), iterations=2)
    # 获取所有检测框
    contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print(len(contours))

    for c in contours:
        # 计算矩形框的面积
        area = cv2.contourArea(c)
        if area >  300:
            # 获取矩形框边界坐标
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("detection", frame)
    cv2.imshow("back", dilated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
camera.release()


## 帧间差分法

**注意：**这种方法十分受光线变化影响

In [1]:
import cv2
import numpy as np

camera = cv2.VideoCapture(0)  # 参数0表示第一个摄像头
# 判断视频是否打开
if camera.isOpened():
    print('Open')
else:
    print('摄像头未打开')

# 测试用,查看视频size
size = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('size:' + repr(size))

es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 4))
kernel = np.ones((5, 5), np.uint8)
background = None

while True:
    # 读取视频流
    grabbed, frame_lwpCV = camera.read()
    # 对帧进行预处理，先转灰度图，再进行高斯滤波。
    # 用高斯滤波进行模糊处理，进行处理的原因：每个输入的视频都会因自然震动、光照变化或者摄像头本身等原因而产生噪声。对噪声进行平滑是为了避免在运动和跟踪时将其检测出来。
    gray_lwpCV = cv2.cvtColor(frame_lwpCV, cv2.COLOR_BGR2GRAY)
    gray_lwpCV = cv2.GaussianBlur(gray_lwpCV, (7, 7), 0)

    # 将第一帧设置为整个输入的背景
    if background is None:
        background = gray_lwpCV
        continue
    # 对于每个从背景之后读取的帧都会计算其与背景之间的差异，并得到一个差分图（different map）。
    # 还需要应用阈值来得到一幅黑白图像，并通过下面代码来膨胀（dilate）图像，从而对孔（hole）和缺陷（imperfection）进行归一化处理
    diff = cv2.absdiff(background, gray_lwpCV)
    # 二值化阈值处理
    diff = cv2.threshold(diff, 148, 255, cv2.THRESH_BINARY)[1]
    # 形态学膨胀
    diff = cv2.dilate(diff, es, iterations=2)
    # 显示矩形框
    contours, hierarchy = cv2.findContours(diff.copy(), cv2.RETR_EXTERNAL,
                                           cv2.CHAIN_APPROX_SIMPLE)
    # 该函数计算一幅图像中目标的轮廓
    for c in contours:
        # 小面积过滤，对于光照不变和噪声低的摄像头可不设定轮廓最小尺寸的阈值
        if cv2.contourArea(c) < 400:
            continue
        (x, y, w, h) = cv2.boundingRect(c)  # 该函数计算矩形的边界框
        cv2.rectangle(frame_lwpCV, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow('contours', frame_lwpCV)
    cv2.imshow('dis', diff)

    key = cv2.waitKey(1) & 0xFF
    # 按'q'健退出循环
    if key == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

Open
size:(640, 480)


# 目标跟踪

## 手动选择区域

[参考](https://www.jianshu.com/p/e0b12a5e9f71)

In [ ]:
import sys
from imutils.video import VideoStream
from imutils.video import FPS
import cv2

def main(video, algorithm):
    """ 
    :param video: 待处理的视频文件
    :param algorithm: 指定OpenCV中的跟踪算法
    """

    major_ver, minor_ver, subminor_ver = cv2.__version__.split('.')

    # 根据opencv的不同版本，创建跟踪器
    if int(major_ver) < 3:
        tracker = a(algorithm)
    else:
        if algorithm == 'BOOSTING':
            tracker = cv2.TrackerBoosting_create()
        if algorithm == 'MIL':
            tracker = cv2.TrackerMIL_create()
        if algorithm == 'KCF':
            tracker = cv2.TrackerKCF_create()
        if algorithm == 'TLD':
            tracker = cv2.TrackerTLD_create()
        if algorithm == 'MEDIANFLOW':
            tracker = cv2.TrackerMedianFlow_create()
        if algorithm == 'GOTURN':
            tracker = cv2.TrackerGOTURN_create()
        if algorithm == "CSRT":
            tracker = cv2.TrackerCSRT_create()
        if algorithm == 'MOSSE':
            tracker = cv2.TrackerMOSSE_create()

    # 读取视频文件
    video_cap = cv2.VideoCapture(video)

    # 检查视频文件是否被正确打开
    if not video_cap.isOpened():
        print("Open video failed.")
        sys.exit()

    # 读取第一帧数据
    ok, frame = video_cap.read()
    if not ok:
        print('Read video file failed.')
        sys.exit()

    # 手动选择关注的区域
    bbox = cv2.selectROI("Frame", frame, showCrosshair=False)
    cv2.destroyWindow('Frame')

    # 初始化跟踪器
    ok = tracker.init(frame, bbox)
    
    while True:
        # 读取下一帧数据
        ok, frame = video_cap.read()
        if not ok:
            break

        # 开始计时器
        timer = cv2.getTickCount()

        # 更新跟踪器
        ok, bbox = tracker.update(frame)

        # 计算fps
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

        # 画出 bounding box
        if ok:
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        # 显示
        cv2.putText(frame, algorithm + " Tracker", (100, 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)
        cv2.putText(frame, "FPS : " + str(int(fps)), (100, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50, 170, 50), 2)

        cv2.imshow("Tracking", frame)
        
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord("s"):
        # select the bounding box of the object we want to track (make
        # sure you press ENTER or SPACE after selecting the ROI)
        bbox = cv2.selectROI("Frame", frame, fromCenter=False,
                               showCrosshair=True)

        # start OpenCV object tracker using the supplied bounding box
        # coordinates, then start the FPS throughput estimator as well
        tracker.init(frame, bbox)

    # 接收到q键，退出循环
    elif key== ord('q'):
        sys.exit()


if __name__ == '__main__':
    main(0, 'KCF')

# 内置人脸检测

In [5]:
import os
import cv2


def detectAndDisplay(frame):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)

    #-- Detect faces
    faces = face_cascade.detectMultiScale(frame_gray)
    for (x,y,w,h) in faces:
        center = (x + w//2, y + h//2)
        frame = cv2.ellipse(frame, center, (w//2, h//2), 0, 0, 300, (255, 0, 255), 4)

        faceROI = frame_gray[y:y+h,x:x+w]
        #-- In each face, detect eyes
        eyes = eyes_cascade.detectMultiScale(faceROI)
        for (x2,y2,w2,h2) in eyes:
            eye_center = (x + x2 + w2//2, y + y2 + h2//2)
            radius = int(round((w2 + h2)*0.25))
            frame = cv2.circle(frame, eye_center, radius, (255, 0, 0 ), 4)

    cv2.imshow('Capture - Face detection', frame)


face_cascade_name = os.path.join(data_path, 'haarcascade_frontalface_alt.xml')
eyes_cascade_name = os.path.join(data_path, 'haarcascade_eye_tree_eyeglasses.xml')

face_cascade = cv2.CascadeClassifier()
eyes_cascade = cv2.CascadeClassifier()

#-- 1. Load the cascades
if not face_cascade.load(cv2.samples.findFile(face_cascade_name)):
    print('--(!)Error loading face cascade')
    exit(0)
if not eyes_cascade.load(cv2.samples.findFile(eyes_cascade_name)):
    print('--(!)Error loading eyes cascade')
    exit(0)

camera_device = 0
#-- 2. Read the video stream
cap = cv2.VideoCapture(camera_device)
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        print('--(!) No captured frame -- Break!')
        break

    detectAndDisplay(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

NameError: name 'camera' is not defined

# [匹配图像位置](https://blog.csdn.net/ns2250225/article/details/60334176/)

In [6]:
import cv2
import aircv as ac

In [7]:
cd C:\Users\Administrator\Desktop

C:\Users\Administrator\Desktop


In [ ]:
# print circle_center_pos
def draw_circle(img, pos, circle_radius, color, line_width):
    cv2.circle(img, pos, circle_radius, color, line_width)
    cv2.imshow('objDetect', imsrc) 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

imsrc = ac.imread('bg.jpg')
imobj = ac.imread('obj.png')
imsrc.shape, imobj.shape

In [ ]:
# find the match position
pos = ac.find_template(imsrc, imobj)
pos

In [ ]:
circle_center_pos = tuple(map(int, pos['result']))

circle_radius = 50
color = (0, 255, 0)
line_width = 10

# draw circle
draw_circle(imsrc, circle_center_pos, circle_radius, color, line_width)

In [ ]:
img = cv2.imread('bg.jpg')
img.shape

In [ ]:
pos = [(744, 405), (744, 495), (842, 405), (842, 495)]
obj = img[405:495, 744:842 ,:]

In [ ]:
Image.fromarray(obj)